In [ ]:
!pip install minimalmodbus

In [ ]:
import minimalmodbus
import time

# Define the serial port parameters

In [ ]:
port = 'COM3'  # Change this to the actual serial port on your system
baudrate = 9600
stopbits = 1
bytesize = 8
parity = minimalmodbus.serial.PARITY_NONE

# Create a Modbus instrument object

In [ ]:
instrument = minimalmodbus.Instrument(port, slaveaddress=1)  # Replace '1' with your device's slave address

# Set serial communication parameters

In [ ]:
instrument.serial.baudrate = baudrate
instrument.serial.bytesize = bytesize
instrument.serial.stopbits = stopbits
instrument.serial.parity = parity

# Function to read a range of coil status

In [ ]:
def read_coil_status(start_address, num_coils):
    try:
        return instrument.read_bits(start_address, num_coils)
    except IOError:
        print("Communication error. Check your connection.")
        return [False] * num_coils  # Return false values in case of communication error

# Function to write a coil status (e.g., for blinking coil 0)

In [ ]:
def write_coil_status(address, value):
    try:
        instrument.write_bit(address, value)
    except IOError:
        print("Communication error. Check your connection.")

# Main loop

In [ ]:
try:
    # Main loop
    while True:
        # Read coil status for coils 0 to 7
        coil_status_0_to_7 = read_coil_status(0, 8)
        
        # Blink coil 0 at every 1 second
        write_coil_status(0, not coil_status_0_to_7[0])  # Toggle the value of coil 0
        
        # Check if coil 4 is low and turn on coil 3, or turn it off if coil 4 is high
        try:
            if not coil_status_0_to_7[4]:
                write_coil_status(3, True)  # Turn on coil 3
            else:
                write_coil_status(3, False)  # Turn off coil 3
        except IOError:
            print("Communication error when controlling coil 3.")
        
        # Wait for 1 second before the next iteration
        time.sleep(1)

except KeyboardInterrupt:
    # Stop the loop when a KeyboardInterrupt (e.g., Ctrl+C) is detected
    print("Loop stopped due to user interruption.")